## Machine Learning Models

### Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict

pd.pandas.set_option('display.max_columns', None)

In [4]:
dataset=pd.read_csv('X_train.csv')
dataset_test=pd.read_csv('X_test.csv')

In [5]:
print (len(dataset_test.columns))
print (len(dataset.columns))

255
255


### Defining predictor and target variables

In [6]:
X=[]
for feature in dataset.columns:
    if feature not in ['Id','SalePrice','YrSold','SalePrice1']:
        X.append(feature)

Y='SalePrice'

### Ensemble regression models with weighted predictions

1. XGBRegrssor (50%)
2. Lasso (25%)
3. Ridfe (25%)

In [11]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import Lasso, Ridge
import statistics

In [14]:
x=20
result=[]
for i in range(x):
    X_train, X_test, y_train, y_test = train_test_split(dataset[X],dataset[Y],test_size=0.15, random_state=i)
    
   
    #reg4=AdaBoostRegressor(n_estimators=1000, loss='square', learning_rate=0.02)
    #reg4.fit(X_train, y_train)
    #ypred4=reg4.predict(X_test)
    #mse4= mean_squared_error(y_test,ypred4)
    
    reg3=XGBRegressor(eval_metric='rmse',max_depth=5,booster='gbtree', n_estimators=1000, learning_rate=0.02)

    reg3.fit(X_train, y_train)
    ypred3=reg3.predict(X_test)
    mse3= mean_squared_error(y_test,ypred3)
    
    reg2=Ridge(alpha=0.001, random_state=0)
    reg2.fit(X_train,y_train)
    ypred2=reg2.predict(X_test)
    mse2= mean_squared_error(y_test,ypred2)
    
    reg1=Lasso(alpha=0.001, random_state=0)
    reg1.fit(X_train,y_train)
    ypred1=reg1.predict(X_test)
    mse1= mean_squared_error(y_test,ypred1)
    
   # reg=LinearRegression()
   # reg.fit(X_train[X1],y_train)
    
    ypred_all=(0.5*ypred3)+(0.25*ypred2)+(0.25*ypred1)
    mse_all=mean_squared_error(y_test,ypred_all)
    result.append(mse_all**(0.5))
    print ('XGB:', mse3**(0.5),'Ridge:',mse2**(0.5),'Lasso:',mse1**(0.5),'Combined:',mse_all**(0.5))

print (statistics.mean(result))

XGB: 0.12514044626249182 Ridge: 0.1446361995306658 Lasso: 0.12525502558540189 Combined: 0.12188440282600553
XGB: 0.1197609707640467 Ridge: 0.13130724713151745 Lasso: 0.12194710129571555 Combined: 0.11512083014580904
XGB: 0.1371216228166307 Ridge: 0.14236002297860395 Lasso: 0.1229045930279637 Combined: 0.12037233904093576
XGB: 0.11250658819642306 Ridge: 0.18025300164018068 Lasso: 0.14261833088575018 Combined: 0.12254453596241714
XGB: 0.10262592597692069 Ridge: 0.11111405509645199 Lasso: 0.10011866328623416 Combined: 0.0905856497213898
XGB: 0.1362973714774195 Ridge: 0.13921790292401257 Lasso: 0.13070262104843083 Combined: 0.12493017369697976
XGB: 0.12775825346912406 Ridge: 0.13006944992658673 Lasso: 0.12800914447971418 Combined: 0.11949242935553792
XGB: 0.11486340810014566 Ridge: 0.12170963468075356 Lasso: 0.12060762971268048 Combined: 0.10972182963618846
XGB: 0.09840181299874562 Ridge: 0.10225809602076823 Lasso: 0.10047472271508738 Combined: 0.09054257566550904
XGB: 0.09620900280695666 

In [439]:
reg3=XGBRegressor(eval_metric='rmse',max_depth=5,booster='gbtree', n_estimators=1000, learning_rate=0.02)
reg3.fit(dataset[X],dataset[Y])
ypred3=reg3.predict(dataset_test[X])
reg2=Ridge(alpha=0.001, random_state=0)
reg2.fit(dataset[X],dataset[Y])
ypred2=reg2.predict(dataset_test[X])
reg1=Lasso(alpha=0.001, random_state=0)
reg1.fit(dataset[X],dataset[Y])
ypred1=reg1.predict(dataset_test[X])

ypred_all=(0.5*ypred3)+(0.25*ypred2)+(0.25*ypred1)
ypred_all=np.exp(ypred_all)
dataset_test['SalePrice']=ypred_all
data=dataset_test[['Id','SalePrice']]

data.to_csv('submission2.csv', index=False)